<a href="https://colab.research.google.com/github/SiHanWang2Developer/data/blob/master/mypointrend%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install detectron2

查询GPU运行状态

In [0]:
!/opt/bin/nvidia-smi

Fri Mar 27 14:59:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |    802MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# install dependencies: 安装依赖项
# (use +cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

In [0]:
# install detectron2: 安装detectron2 
!pip install detectron2==0.1.1 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
# clone the repo to access PointRend code.  克隆detectron2获取pointrend代码Use the same version as the installed detectron2
# !git clone --branch v0.1.1 https://github.com/facebookresearch/detectron2 detectron2_repo

连接Google Drive 
加载 数据集 和 detectron2_repo

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
# 您可能需要在此之前重新启动runtime，以使安装生效
# 一些基本设置：
# 设置detectron2记录器
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries导入一些常见的库
import numpy as np
import cv2
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities导入一些常见的detectron2包
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
coco_metadata = MetadataCatalog.get("coco_2017_val")

# import PointRend project  导入PointRend项目
import sys; sys.path.insert(1, "detectron2_repo/projects/PointRend")
import point_rend

## 运行预训练的PointRend模型

//   ![替代文字](https://)我们首先从COCO数据集中下载图像：

In [0]:
!wget http://images.cocodataset.org/val2017/000000005477.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
cv2_imshow(im)

加载私人数据集 下载图像

In [0]:
# !wget http://images.cocodataset.org/val2017/000000005477.jpg -q -O input.jpg
input.jpg="./drive/My Drive/......[具体路径]"
im = cv2.imread("./input.jpg")
cv2_imshow(im)

然后，我们创建一个detectron2 config 和一个detectron2 DefaultPredictor以在该mask上运行推理。

首先，我们使用标准的Mask R-CNN模型进行预测。

In [0]:
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model为此模型设置阀值
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# mask_rcnn_predictor = DefaultPredictor(cfg)
# mask_rcnn_outputs = mask_rcnn_predictor(im)

现在，我们加载一个PointRend模型并显示其预测。

In [0]:
cfg = get_cfg()
# Add PointRend-specific config添加PointRend特定的配置
point_rend.add_pointrend_config(cfg)
# Load a config from file从文件加载配置
cfg.merge_from_file("[具体路径]detectron2_repo/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model为此模型设置阀值
# Use a model from PointRend model zoo:使用PointRend model zoo中的模型 https://github.com/facebookresearch/detectron2/tree/master/projects/PointRend#pretrained-models
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/detectron2/PointRend/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco/164955410/model_final_3c3198.pkl"
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [0]:
# Show and compare two predictions: 显示并比较两个预测
# v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
# mask_rcnn_result = v.draw_instance_predictions(mask_rcnn_outputs["instances"].to("cpu")).get_image()
v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
point_rend_result = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()
print("Mask R-CNN with PointRend (top)     vs.     Default Mask R-CNN (bottom)")
# cv2_imshow(np.concatenate((point_rend_result, mask_rcnn_result), axis=0)[:, :, ::-1])
cv2_imshow((（point_rend_result）axis=0）[:, :, ::-1])



---



---



## 训练PointRend模型

In [0]:
import os
import torch

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
# from detectron2.config import get_cfg
# from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    CityscapesEvaluator,
    COCOEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    verify_results,
)


from point_rend import add_pointrend_config  #projects.PointRend.point_rend.config

In [0]:
class Trainer(DefaultTrainer): #from detectron2.engine import DefaultTrainer 
    """
    We use the "DefaultTrainer" which contains a number pre-defined logic for standard training workflow.
    我们使用“ DefaultTrainer”，其中包含一些 用于标准训练工作流程的 预定义逻辑。
    They may not work for you, especially if you are working on a new research project. 
    它们可能对您不起作用，特别是如果您正在从事新的研究项目。
    In that case you can use the cleaner "SimpleTrainer", or write your own training loop.
    在这种情况下，您可以使用cleaner“ SimpleTrainer”，也可以编写自己的训练循环
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        为给定的数据集创建评估器。
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        这将使用 与每个内置数据集关联的 特殊元数据（metadata）“!!!evaluator_type!!!”。
        For your own dataset, you can simply create an evaluator manually in your script and do not have to worry about the hacky if-else logic here.
        对于您自己的数据集，您只需在脚本中手动创建一个 评估器 ，而不必担心这里的hacky if-else逻辑
        对于您自己的数据集，您只需在脚本中手动创建一个 评估器
        只需在脚本中手动创建一个 评估器
        手动创建一个 评估器
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type 
        #与每个内置数据集关联的 特殊元数据（metadata）“!!!evaluator_type!!!”

       """   
       我的数据集evaluator_type='coco',！！！！！！！！！！！！！！！！！！！！
Metadata(
  evaluator_type='coco', 
  image_root='./drive/My Drive/pic500_28class/images', 
  json_file='./drive/My Drive/pic500_28class/train.json', name='wzb', 
  thing_classes=['piezhe', 'heng', 'hengzhewangou', 'pie', 'na', 'shuwangou', 'henggou','hengzheti', 'shuwan',
       'shugou', 'hengzhegou', 'hengzhezhezhegou', 'hengpie', 'shu', 'shuzhezhegou', 'dian', 'wangou', 'ti', 'shuti', 
       'shuzhe', 'wogou', 'hengzhe', 'xiegou', 'hengzhezhepie', 'hengzhewan', 'piedian', 'hengzhepie', 'hengxiegou' ], 
  thing_dataset_id_to_contiguous_id={1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27})
       """
       
       if evaluator_type == "lvis":
            return LVISEvaluator(dataset_name, cfg, True, output_folder)
        if evaluator_type == "coco":#我的数据集类型是coco
            return COCOEvaluator(dataset_name, cfg, True, output_folder)
        if evaluator_type == "cityscapes":
            assert (
                torch.cuda.device_count() >= comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesEvaluator(dataset_name)
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        if len(evaluator_list) == 1:
            return evaluator_list[0]
        return DatasetEvaluators(evaluator_list)





Create configs and perform basic setups.创建配置configs并执行基本设置 basic setups

In [0]:

def setup(args):
    """
    Create configs and perform basic setups.创建配置并执行基本设置
    """
    cfg = get_cfg()
    add_pointrend_config(cfg)
    cfg.merge_from_file(args.config_file)#.merge_from_file
    cfg.merge_from_list(args.opts)#.merge_from_list
    cfg.freeze()
    default_setup(cfg, args)#来自detectron2.engine.default.py文件中的default_setup：在工作开始时执行一些基本的通用设置，
    return cfg

In [0]:
def main(args):
    cfg = setup(args)

    if args.eval_only:
        model = Trainer.build_model(cfg)
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = Trainer.test(cfg, model)
        if comm.is_main_process():
            verify_results(cfg, res)
        return res

    trainer = Trainer(cfg)  #Trainer(DefaultTrainer):
    trainer.resume_or_load(resume=args.resume)
    return trainer.train()

In [0]:
if __name__ == "__main__":
    args = default_argument_parser().parse_args()  ##来自detectron2.engine.default.py文件中的default__parser() 
    print("Command Line Args:", args)
    launch(  #from detectron2.engine import  launch
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )

In [0]:
# To train a model with 1 GPUs run:
cd /path/to/detectron2/projects/PointRend
python train_net.py --config-file configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_1x_coco.yaml --num-gpus 1

In [0]:
# Model evaluation can be done similarly:
cd /path/to/detectron2/projects/PointRend
python train_net.py --config-file configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_1x_coco.yaml --eval-only MODEL.WEIGHTS /path/to/model_checkpoint
